Data load, set your target and unsused_features

train data and valida data would be 75% and 25% of train data.

In [2]:
from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score

train_data= pd.read_csv('train.mor.csv')
test_data= pd.read_csv('test.mor.csv')
#df = pd.read_csv('./data/my.nih.norm.csv')
#df1 = pd.read_csv('./data/abcd_tbss01.csv')


In [2]:
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)
test_data_processed= test_data.reset_index(drop=True)
train_data_processed = train_data.reset_index(drop=True)

In [29]:

target='sex'
unused_feat = ['Set', 'subjectkey', 'race.ethnicity', 'abcd_site', 'Unnamed: 0']


In [4]:

features = [ col for col in train_data_processed.columns if col not in unused_feat + [target]] 

if "Set" not in train_data_processed.columns:
    train_data_processed["Set"] = np.random.choice(["train", "valid"] , p =[.75, .25], size=(train_data_processed.shape[0],))

train_indices = train_data_processed[train_data_processed.Set=="train"].index
valid_indices = train_data_processed[train_data_processed.Set=="valid"].index



In [5]:

X_train = train_data_processed[features].values[train_indices]
y_train = train_data_processed[target].values[train_indices]
X_valid = train_data_processed[features].values[valid_indices]
y_valid = train_data_processed[target].values[valid_indices]
X_test = test_data_processed[features].values
y_test = test_data_processed[target].values

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ",y_train.shape)
print("X_valid.shape: ", X_valid.shape)
print("y_valid.shape: ",y_valid.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ",y_test.shape)

X_train.shape:  (5800, 1043)
y_train.shape:  (5800,)
X_valid.shape:  (2024, 1043)
y_valid.shape:  (2024,)
X_test.shape:  (1957, 1043)
y_test.shape:  (1957,)


In [6]:
print("X_train: ", X_train)
print("y_train: ",y_train)
print("X_valid: ", X_valid)
print("y_valid: ",y_valid)
print("X_test: ", X_test)
print("y_test: ",y_test)

X_train:  [[ 0.15890019  2.34331573  2.5119754  ...  0.68325767  0.78925124
   0.68887924]
 [-0.51212116 -0.03159145 -0.53800352 ... -0.38710003 -0.22538113
  -0.11662927]
 [-0.61728122 -0.66843806 -0.26238929 ... -0.2840919   0.22571588
  -0.35856821]
 ...
 [-0.10650378  1.13596068 -0.69289416 ...  0.34257862  0.46276225
  -0.06168219]
 [-0.87767756 -0.62200133  0.48473026 ... -1.16414248 -0.64952149
  -0.41393306]
 [ 0.69972336  1.36151053  0.48700807 ...  0.00965159  0.96610002
   1.14473944]]
y_train:  [2 1 1 ... 1 2 1]
X_valid:  [[ 0.09880872  0.67822718  1.19768277 ...  0.05926673  0.30079841
   0.54571449]
 [-0.8876928  -0.24387365 -1.24184481 ... -0.89317866 -1.34036613
  -0.93804441]
 [-1.00787573 -0.58219842 -1.3694018  ...  0.09256084 -0.34782674
  -0.05905194]
 ...
 [ 0.21899165  0.82417119  2.07691493 ... -0.4489326  -0.61850594
  -1.10992749]
 [-1.48860744 -0.97359373 -1.59718216 ... -0.80068039 -0.81380869
  -0.37380318]
 [-1.02790622  0.17405694  1.67602151 ...  0.86346

In [7]:
# Augmented
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
# Store maximum auc
max_auc = 0
# Store maximum hypterparameter set
max_hy = []
# define hyperparameter space
n_ = [4,8,16]
lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
w_ = [0.01, 0.001, 0.0001]
g_ = [0.95, 0.99, 0.9]
ss_ = [10, 20, 30]
all_ = [n_, lr_, w_, g_, ss_]
h_space = [s for s in itertools.product(*all_)]
# Start training


for hy in tqdm(h_space):
    clf = TabNetClassifier(n_a = hy[0],
                           n_d = hy[0],
                           optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                           scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                           scheduler_fn=torch.optim.lr_scheduler.StepLR,
                           verbose=0
                           )
    clf.fit(X_train, y_train, X_valid, y_valid, patience=50, batch_size=256, max_epochs=200)
    preds_acc = clf.predict(X_test)
    preds_prob = clf.predict_proba(X_test)
    test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=y_test)
    test_acc = accuracy_score(preds_acc, y_test)
    print("FINAL TEST SCORE", test_auc, test_acc)
    if test_auc>max_auc:
        max_hy = hy
        max_auc = test_auc


Device used : cuda


KeyboardInterrupt: 

(4, 0.02, 0.01, 0.95, 10) Test AUC: 0.9348476454293627 
(4, 0.02, 0.01, 0.9, 20) Test AUC: 0.935661945330058 

Do it with best hyperparameter 

In [24]:
target= 'sex'
unused_feat = ['fluidcomp', 'Set', 'subjectkey', 'race.ethnicity', 'abcd_site', 'Unnamed: 0']


In [25]:

X_train = train_data_processed[features].values[train_indices]
y_train = train_data_processed[target].values[train_indices]
X_valid = train_data_processed[features].values[valid_indices]
y_valid = train_data_processed[target].values[valid_indices]
X_test = test_data_processed[features].values
y_test = test_data_processed[target].values

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ",y_train.shape)
print("X_valid.shape: ", X_valid.shape)
print("y_valid.shape: ",y_valid.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ",y_test.shape)

X_train.shape:  (5775, 1043)
y_train.shape:  (5775,)
X_valid.shape:  (1870, 1043)
y_valid.shape:  (1870,)
X_test.shape:  (1923, 1043)
y_test.shape:  (1923,)


In [26]:
hy = (4, 0.02, 0.01, 0.9, 20)
clf = TabNetClassifier(n_a = hy[0],
                       n_d = hy[0],
                       optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                       scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       verbose=0
                           )
clf.fit(X_train, y_train, X_valid, y_valid, patience=50, batch_size=256, max_epochs=200)
preds_acc = clf.predict(X_test)
preds_prob = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=y_test)
test_acc = accuracy_score(preds_acc, y_test)
print("FINAL TEST SCORE", test_auc, test_acc)

Device used : cuda


NameError: name 'accuracy_score' is not defined

find feature importance 

In [ ]:
def feature(num):
    importance =clf.feature_importances_
    plt.plot(importance)

    labels_importance=importance.argsort()[::-1]

    importance_sort = np.sort(importance)[::-1]

    feat_name_sort=test_data_processed[features].columns[labels_importance]
    important_features = pd.DataFrame() 


    for i in range (num):
        feature = pd.DataFrame([[feat_name_sort[i],importance_sort[i]]], columns = ['feature name', 'ratio'])
        important_features=pd.concat([important_features,feature])

    print(important_features.reset_index(drop=True))

In [ ]:
feature(5)

In [ ]:
importance =clf.feature_importances_
plt.plot(importance)

labels_importance=importance.argsort()[::-1]

importance_sort = np.sort(importance)[::-1]

feat_name_sort=test_data_processed[features].columns[labels_importance]
important_features = pd.DataFrame() 


for i in range (5):
    feature = pd.DataFrame([[feat_name_sort[i],importance_sort[i]]], columns = ['feature name', 'ratio'])
    important_features=pd.concat([important_features,feature])

important_features.reset_index(drop=True)

In [27]:
saving_path_name = "./sexclassification_tabnet_model_mor_1"
saved_filepath = clf.save_model(saving_path_name)

print("saved_filepath: ", saved_filepath)

Successfully saved model at ./sexclassification_tabnet_model_mor_1.zip
saved_filepath:  ./sexclassification_tabnet_model_mor_1.zip


In [28]:
loaded_clf = TabNetRegressor()
loaded_clf.load_model(saved_filepath)


Device used : cuda
Device used : cuda


In [5]:

train_mor=pd.read_csv ('mor_train_with_NIH.csv')
test_mor =pd.read_csv ('mor_test_with_NIH.csv')
train_NIH =pd.read_csv ('train_with_NIH.csv')
test_NIH = pd.read_csv ('test_with_NIH.csv')


In [6]:
train_mor=train_mor.assign(fluidcomp=train_NIH['nihtbx_fluidcomp_uncorrected'].tolist())

In [7]:
test_mor=test_mor.assign(fluidcomp=test_NIH['nihtbx_fluidcomp_uncorrected'].tolist())

In [8]:
test_mor

,Unnamed: 0,subjectkey,sex,race.ethnicity,abcd_site,lh_bankssts_area._.1,lh_caudalanteriorcingulate_area._.1,lh_caudalmiddlefrontal_area._.1,lh_cuneus_area._.1,lh_entorhinal_area._.1,...,wm.rh.transversetemporal._.18,wm.rh.insula._.18,Left.UnsegmentedWhiteMatter._.18,Right.UnsegmentedWhiteMatter._.18,lhCerebralWhiteMatterVol._.18,rhCerebralWhiteMatterVol._.18,CerebralWhiteMatterVol._.18,MaskVol._.18,EstimatedTotalIntraCranialVol._.18,fluidcomp
0,1,NDAR_INV007W6H7B,1,1,22,1.711263,1.016552,2.038192,0.204396,1.060413,...,1.523832,0.516604,1.493272,1.668963,2.052498,1.981025,2.018829,2.074332,1.368216,0.859540
1,6,NDAR_INV00LH735Y,1,3,3,0.874990,-0.018324,0.368562,-0.770356,0.001715,...,-0.692763,-1.209858,-0.107813,-0.292801,-0.245770,-0.274859,-0.260597,0.181936,0.456559,-1.447688
2,12,NDAR_INV00X2TBWJ,2,3,14,1.145402,0.140888,-0.599504,-1.486840,1.089027,...,-0.803677,-0.620718,-0.772871,-0.895455,-0.356828,-0.457092,-0.407425,-0.008230,0.365968,-0.197940
3,18,NDAR_INV01AJ15N9,1,1,16,0.078778,-0.482691,0.382229,1.312447,1.680372,...,-1.153354,-1.333109,1.398845,1.132769,1.303689,1.250960,1.278631,1.315186,1.157997,-0.390209
4,20,NDAR_INV01ELX9L6,2,2,4,-1.022899,-0.668438,0.368562,-0.316305,-1.400345,...,0.354574,-1.473899,-1.399160,-1.540223,-1.198914,-1.276079,-1.238815,-1.427905,-1.560898,-0.967016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1918,9755,NDAR_INVZX24TGXN,1,1,9,-0.191633,0.107719,0.250117,0.133581,1.365623,...,1.356191,1.675921,0.209144,0.040799,0.937163,0.867282,0.903132,1.330666,1.334954,0.571136
1919,9756,NDAR_INVZXC2YRV3,1,1,10,1.480912,0.200592,0.876512,-0.932814,-0.713622,...,-0.015423,0.158228,-0.332553,-0.645636,0.324504,0.169126,0.247007,0.438227,0.336904,-0.005671
1920,9766,NDAR_INVZZ1MNPK5,2,5,8,-0.296793,-0.648537,-0.127999,-0.832840,-0.894841,...,-0.697843,-1.031239,-0.178276,-0.082675,-0.955257,-0.949396,-0.953314,-1.421635,-1.280167,2.013153
1921,9771,NDAR_INVZZ81LEEV,1,2,11,-0.426992,-0.336747,0.006392,1.987275,-1.142824,...,-2.000029,1.074646,-0.002386,-0.148565,0.181019,0.157140,0.169245,0.149647,-0.059344,-0.582478


In [9]:
train_data = train_mor.fillna(0)
test_data = test_mor.fillna(0)
test_data_processed= test_data.reset_index(drop=True)
train_data_processed = train_data.reset_index(drop=True)

In [10]:

target= 'fluidcomp'
unused_feat = ['sex', 'Set', 'subjectkey', 'race.ethnicity', 'abcd_site', 'Unnamed: 0']


In [11]:

features = [ col for col in train_data_processed.columns if col not in unused_feat + [target]] 

if "Set" not in train_data_processed.columns:
    train_data_processed["Set"] = np.random.choice(["train", "valid"] , p =[.75, .25], size=(train_data_processed.shape[0],))

train_indices = train_data_processed[train_data_processed.Set=="train"].index
valid_indices = train_data_processed[train_data_processed.Set=="valid"].index



In [29]:

X_train = train_data_processed[features].values[train_indices]
y_train = train_data_processed[target].values[train_indices].reshape(-1, 1)
X_valid = train_data_processed[features].values[valid_indices]
y_valid = train_data_processed[target].values[valid_indices].reshape(-1, 1)
X_test = test_data_processed[features].values
y_test = test_data_processed[target].values.reshape(-1, 1)

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ",y_train.shape)
print("X_valid.shape: ", X_valid.shape)
print("y_valid.shape: ",y_valid.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ",y_test.shape)

X_train.shape:  (5775, 1043)
y_train.shape:  (5775, 1)
X_valid.shape:  (1870, 1043)
y_valid.shape:  (1870, 1)
X_test.shape:  (1923, 1043)
y_test.shape:  (1923, 1)


In [31]:
"""
hy = (4, 0.02, 0.01, 0.9, 20)
clf = TabNetClassifier(n_a = hy[0],
                       n_d = hy[0],
                       optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                       scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       verbose=0
                           )
"""
from sklearn.metrics import mean_squared_error

loaded_clf.fit(X_train, y_train, X_valid, y_valid, patience=50, batch_size=256, max_epochs=200)

preds = clf.predict(X_test)

y_true = y_test

test_score = mean_squared_error(y_pred=preds, y_true=y_true)

print(f"BEST VALID SCORE FOR  : {-clf.best_cost}")
print(f"FINAL TEST SCORE FOR : {test_score}")


BEST VALID SCORE FOR  : 0.9386643786643787
FINAL TEST SCORE FOR : 0.17212688507540302


In [32]:
from sklearn.metrics import mean_squared_error

clf = TabNetRegressor()

clf.fit(X_train, y_train, X_valid, y_valid, patience=50, batch_size=256, max_epochs=200)

preds = clf.predict(X_test)

y_true = y_test

test_score = mean_squared_error(y_pred=preds, y_true=y_true)

print(f"BEST VALID SCORE FOR  : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR : {test_score}")


Device used : cuda
Will train until validation stopping metric hasn't improved in 50 rounds.
---------------------------------------
| EPOCH |  train  |   valid  | total time (s)
| 1     | -1.00279 |  -0.41659 |   1.3       
| 2     | -0.26362 |  -0.28629 |   2.6       
| 3     | -0.21511 |  -0.24761 |   3.9       
| 4     | -0.19791 |  -0.21979 |   5.2       
| 5     | -0.19947 |  -0.21389 |   6.5       
| 6     | -0.19349 |  -0.20340 |   7.9       
| 7     | -0.19838 |  -0.20587 |   9.1       
| 8     | -0.19181 |  -0.20586 |   10.5      
| 9     | -0.19287 |  -0.20167 |   11.8      
| 10    | -0.18865 |  -0.20289 |   13.1      
| 11    | -0.18662 |  -0.19664 |   14.5      
| 12    | -0.18520 |  -0.19847 |   15.8      
| 13    | -0.18482 |  -0.19703 |   17.1      
| 14    | -0.18497 |  -0.21056 |   18.5      
| 15    | -0.19004 |  -0.19199 |   19.8      
| 16    | -0.18181 |  -0.20457 |   21.1      
| 17    | -0.18301 |  -0.19199 |   22.4      
| 18    | -0.17833 |  -0.19234 |   23.7